In [1]:
#Import required packages
import requests
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import time
import os
import pickle

#### The following Cell contains a function that will scrape bitcoin talk forum using selenium webdriver for links to individual posts that users generate to discuss bitcoin news

In [2]:
#Set chromedriver executable path. 
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#Define function to extract forum discussion links. 
def get_bitcoin_news_links(url):
    """
    A function utilizing selenium to scrape the links to individual posts within a forum table.
    """
    driver = webdriver.Chrome(chromedriver)
    print('Scraping Bitcoin News Links\n')
    
    #Empty list to store forum post links
    a_elements = []
    
    #The forum posting links begin with 77.'# of posts ' and increment by 40. Will loop through to obtain last ~34,000 links
    i = 0
    while i <= 34160:
        #Establish link
        bitcoinnews_url=url+str(i)
        
        #Utilize selenium drive to launch link
        driver.get(bitcoinnews_url)
        table_id = driver.find_element(By.XPATH,'//*[@id="bodyarea"]/div[2]/table/tbody') #Define xpath to forum article discussion
        rows = table_id.find_elements_by_tag_name("tr")
    
        for row in rows:
            link = row.find_element(By.TAG_NAME, "a")
            a_elements.append(link.get_attribute('href'))
        i+=40
        
    print('Scraping Done!\n')
    
    return a_elements

In [ ]:
#Define site URL for scraping
url="https://bitcointalk.org/index.php?board=77."

#Scrap News Article Posts. Only run once then comment out, will take approximately 5-10min
forum_subject_links = get_bitcoin_news_links(url)

In [ ]:
#Remove First Few Links that are forum overhead links (example, Privacy Policy, Terms of Agreement, Rules, ect.)
n = 4
del forum_subject_links[:n]

#Remove Access Links
forum_subject_links = [x for x in forum_subject_links if "sort=subject" not in x]

#Save forumn links list
with open('./article_data/forum_subject_links.pkl', 'wb') as f:
    pickle.dump(forum_subject_links, f)

In [4]:
#Store Post Links
with open('./data_frames/forum_subject_links.pkl', 'rb') as file:
     forum_subject_links = pickle.load(file)

#### The following cell defines a function that scrapes the individual posts located above for the news link mentioned in the posts. This was done in parts. Then saved as lists to drive. 

In [26]:
#Define function to extract news article links from individual forum discussions
def get_article_links_from_forumns():
    """
    Function utilizing selenium webdriver to scrape the news article link posted within forum posts. 
    """
    driver = webdriver.Chrome(chromedriver)
    print('Scraping Bitcoin News Links\n')
    
    href_elements = []
    for url in forum_subject_links[15000:30000]:
        try:
            driver.get(url)
            time.sleep(.5)
            post_location = driver.find_element(By.XPATH,'//*[@id="quickModForm"]/table[1]/tbody/tr[1]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/div')
            a_element = post_location.find_element_by_tag_name('a')
            href = a_element.get_attribute('href')
            href_elements.append(href)
        except:
            pass
   

    print('Scraping Done!')
    return href_elements

In [27]:
news_article_links_15000_30000 = get_article_links_from_forumns()

Scraping Bitcoin News Links

Scraping Done!


In [28]:
#Generate empty list to store links. 
news_article_links =[]
# news_article_links.append(news_article_links_0_2000)
# news_article_links.append(news_article_links_4000_10000)
# news_article_links.append(news_article_links_10000_15000)
news_article_links.append(news_article_links_15000_30000)

In [29]:
#Save News Links
with open('./data_frames/news_article_links_15000_30000.pkl', 'wb') as f:
    pickle.dump(news_article_links, f)